In [ ]:
import pandas as pd
import os
import glob
import pycytominer

import CBE_utils as CBE


In [ ]:
import importlib
importlib.reload(CBE)

In [ ]:
input_path = "/media/schmied.christopher/T7 Shield/Datasets/ECBL/processed/"
annotation_dir = "/home/schmied.christopher/FMP_Docs/Projects/eu_os_ecbl_qc/annotation/"

# test
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/test_output/"
annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/test_annotation/"

In [ ]:
pattern = "[A-Z]*_R[1-4].csv"

file_list = glob.glob(input_path + os.sep + '*' + os.sep + '*' + os.sep + pattern, recursive=True)

In [ ]:
for file in file_list:

    ## Choose normalization method
    normalizer = "mad_robustize" 
    
    plate_map_name = os.path.splitext(os.path.basename(file))[0]

    # Check if output file already exists
    filename_norm_data = plate_map_name + "_" + normalizer + ".csv"
    path_norm_data = os.path.dirname(file)
    file_path_norm_data = os.path.join(path_norm_data, filename_norm_data)
    
    if os.path.exists(file_path_norm_data):
        print(f"INFO: Skipping {filename_norm_data} already normalized with {normalizer}")
        continue
    
    try:
        
        dataframe = pd.read_csv(file)
        row_count = dataframe.shape[0]
        
        print(f"INFO: {plate_map_name} has {row_count} rows")
        
        
    except Exception as e:
        
        print(f"Error reading file {plate_map_name}: {e}")   
        
    features = CBE.get_feature_vector(dataframe)
    metadata_dataframe = set(dataframe.columns) - set(features)
        
    ### convert sets to list
    features_list = list(features)
    metadata_list = list(metadata_dataframe)
        
    number_features = len(features_list)
        
    print(f"INFO: {plate_map_name} has {number_features} features")
        
    
    
    dataframe_normalized = pycytominer.normalize(
        
        profiles = dataframe,
        features = features_list,
        meta_features = metadata_list,
        method = normalizer, ### Method to normalize the data
        samples = "Metadata_RoughID == 'DMSO'" # normalization performed on neg. controls
    )
    
    print(f"INFO: Normalized with {normalizer}")
    
    filename_norm_data = plate_map_name + "_" + normalizer + ".csv"
    
    path_norm_data = os.path.dirname(file)
    file_path_norm_data = os.path.join(path_norm_data, filename_norm_data)
    
    dataframe_normalized.to_csv(file_path_norm_data, index=False)
    
    print(f"INFO: Saved {filename_norm_data}")